In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
import importlib
import time
import datetime
import json
import threading
from threading import Event

import shioaji as sj
import login.shioaji_login as shioaji_login
from shioaji import BidAskFOPv1
# from shioaji.constant import Action, OptionRight, StockPriceType, FuturesOrderType, FuturesOCType #選擇權下單，多匯入一個OptionRight常數
importlib.reload(shioaji_login)

import tools.globals as globals
import tools.get_snap_options as snap
import tools.message_log as message_log
import tools.get_simulate_positions as positions
import tools.cover as cover
import tools.contract as contract

In [2]:
def update_config():
    """
    Threading function.
    每秒更新config相關參數
    
    :global param: get_simulation_time
    :global param: simulation_mode
    :global param: simulation_optionright
    :global param: simulation_quantity
    :global param: simulation_action

    :global param: get_cover_time
    :global param: cover_put_strike
    :global param: cover_call_strike
    :global param: cover_quantity

    :return: None
    """

    pre_get_simulation_time = None
    pre_simulation_mode = None
    pre_simulation_optionright = None
    pre_simulation_quantity = None
    pre_simulation_action = None

    pre_get_cover_time = None 
    pre_cover_mode = None
    pre_cover_put_strike = None 
    pre_cover_call_strike = None 
    pre_cover_quantity = None

    while(True):

        with open('config.json') as f:
            config_data = json.load(f)

            globals.get_simulation_time =  datetime.datetime.strptime(config_data['get_simulation_time'], '%H:%M:%S').time()
            globals.get_cover_time =  datetime.datetime.strptime(config_data['get_cover_time'], '%H:%M:%S').time()

            ### set simulation ###
            if config_data['simulation_mode'].lower() == "false":
                globals.simulation_mode = False
            else:
                globals.simulation_mode = True

            if config_data['simulation_optionright'].lower() == "c":
                globals.simulation_optionright = 'C'
            else:
                globals.simulation_optionright = 'P'

            if config_data['simulation_action'].lower() == "buy":
                globals.simulation_action = 'Buy'
            else:
                globals.simulation_action = 'Sell'

            globals.simulation_quantity = config_data['simulation_quantity']
            
            ### set cover ###
            if config_data['cover_mode'].lower() == "false":
                globals.cover_mode = False
            else:
                globals.cover_mode = True

            globals.cover_put_strike = config_data['cover_put_strike']
            globals.cover_call_strike = config_data['cover_call_strike']
            globals.cover_quantity = config_data['cover_quantity']

            ### detect ###
            if(pre_get_simulation_time != globals.get_simulation_time):
                print(f'Get simulation time has been set to {globals.get_simulation_time}')
                pre_get_simulation_time = globals.get_simulation_time

            if(pre_simulation_mode != globals.simulation_mode):
                print(f'Simulation_mode has been set to {globals.simulation_mode}')
                pre_simulation_mode = globals.simulation_mode

            if(pre_simulation_optionright != globals.simulation_optionright):
                print(f'Simulation_optionright has been set to {globals.simulation_optionright}')
                pre_simulation_optionright = globals.simulation_optionright

            if(pre_simulation_action != globals.simulation_action):
                print(f'Simulation_action has been set to {globals.simulation_action}')
                pre_simulation_action = globals.simulation_action

            if(pre_simulation_quantity != globals.simulation_quantity):
                print(f'Simulation_quantity has been set to {globals.simulation_quantity}')
                pre_simulation_quantity = globals.simulation_quantity

            if(pre_get_cover_time != globals.get_cover_time):
                print(f'Get cover time has been set to {globals.get_cover_time}')
                pre_get_cover_time = globals.get_cover_time

            if(pre_cover_mode != globals.cover_mode):
                print(f'Cover mode has been set to {globals.cover_mode}')
                pre_cover_mode = globals.cover_mode

            if(pre_cover_put_strike != globals.cover_put_strike):
                print(f'Cover put strike has been set to {globals.cover_put_strike}')
                pre_cover_put_strike = globals.cover_put_strike

            if(pre_cover_call_strike != globals.cover_call_strike):
                print(f'Cover call strike has been set to {globals.cover_call_strike}')
                pre_cover_call_strike = globals.cover_call_strike

            if(pre_cover_quantity != globals.cover_quantity):
                print(f'Cover quantity has been set to {globals.cover_quantity}')
                pre_cover_quantity = globals.cover_quantity

            time.sleep(1)



In [3]:
def update_snap_options():
    """
    Threading function.
    每秒偵測,若現在時間符合get_simulation_time, 則找尋當下價平 
    且若simulation_mode為True則下模擬單
    
    :global param: get_simulation_time
    :global param: simulation_mode
    :return: None
    """

    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_simulation_time):
            if globals.simulation_mode:
                snap.get_snap_options()
                snap.get_at_the_money_info()
                print(globals.get_simulation_time, "時刻之價平和檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
                
                option_code = globals.at_the_money_code +  snap.get_option_code(globals.simulation_optionright)
                globals.contract = contract.fill_contract(option_code)
                positions.place_simulate_order(globals.simulation_quantity, option_code, globals.simulation_optionright, globals.simulation_action)

        time.sleep(1)

In [4]:
def detect_cover_time():
    """
    Threading function.
    每秒偵測,判斷現在時間符合get_cover_time 符合則交付cover控制
    且若simulation_mode為True則下模擬單
    
    :global param: get_cover_time
    :return: None
    """
    while(True): 
        now = datetime.datetime.now()
        
        if(now.time().replace(microsecond=0) == globals.get_cover_time):
            log_msg = f"A cover time been detected. time: {globals.get_cover_time}\n"
            print(log_msg)
            message_log.write_log(log_msg)
            cover.cover_controller()
        time.sleep(1)

In [5]:
def price_checker():
    """
    每隔15秒會偵測一次最新價平 當目前價平模擬單之成交價虧損達1.4倍則平倉出場
    :global param: positions (list)
    :return: None
    """
    while(True):
        for p in globals.positions:
            if(p[0] == 1):
                askbid = 'bidprice' #要平倉現有買進倉位 因此找最佳賣價
            elif(p[0] == -1):
                askbid = 'askprice' #要平倉現有賣出倉位 因此找最佳買價
            price = cover.update_at_the_money_price(globals.simulation_optionright, askbid)
            condition1 = p[0] == 1 and price <= p[2]*(1/1.4)
            condition2 = p[0] == -1 and price >= p[2]*1.4

            if(condition1 or condition2):
                log_msg = f"A loss stop has been detected. Market price: {price}, buy price: {p[2]}\n"
                print(log_msg)
                message_log.write_log(log_msg)
                cover.cover_controller()

        time.sleep(15)
        

In [6]:
def load_position():
    # 取得Position 
    globals.position = globals.api.list_positions(globals.api.futopt_account)

    if len(globals.position) == 0:
        print("None")

In [7]:
def main():
    """
    Controlled whole program, make app execute.
    
    :global param: api
    """
    # initialize all global variables
    globals.initialize()

    # load_position()

    # log in
    globals.api = shioaji_login.login()
    globals.api.set_order_callback(positions.place_cb)

    # Start update config thread. All config variable will be updated every second.
    update_config_thread = threading.Thread(target = update_config)
    update_config_thread.start()
    time.sleep(1)

    update_snap_options_thread = threading.Thread(target = update_snap_options)
    update_snap_options_thread.start()
    time.sleep(1)

    detect_cover_time_thread = threading.Thread(target = detect_cover_time)
    detect_cover_time_thread.start()
    time.sleep(1)

    
    price_checker_thread = threading.Thread(target = price_checker)
    price_checker_thread.start()
    time.sleep(1)
    
    snap.get_snap_options()
    snap.get_at_the_money_info()
    print("程式始執行之價平和檔位及成交點數和: ", globals.at_the_money_code, globals.at_the_money)
    
    ### 訂閱平倉檔次之bidask ###
    cover_call_code, cover_put_code = cover.get_cover_code()
    print("put: ", cover_put_code, "call: ", cover_call_code)
    cover.subscribe_cover_code(cover_call_code, cover_put_code)
    

    # print(api.Contracts.Options.TX4)



In [8]:
if __name__ == "__main__":
    main() 

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID R124743829
Login status: [FutureAccount(person_id='R124743829', broker_id='F002000', account_id='1697256', signed=True, username='劉庭銘'), Account(account_type=<AccountType.H: 'H'>, person_id='R124743829', broker_id='9A9X', account_id='0150198', username='劉庭銘\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000'), StockAccount(person_id='R124743829', broker_id='9A9X', account_id='0510980', signed=True, username='劉庭銘\u3000\u3000')]
Activating CA at the path C:/sinopac/Sinopac.pfx


Get simulation time has been set to 01:56:00
Simulation_mode has been set to True
Simulation_optionright has been set to P
Simulation_action has been set to Buy
Simulation_quantity has been set to 5
Get cover time has been set to 01:36:00
Cover mode has been set to Fa

In [9]:
globals.txo_weekly_dict[globals.at_the_money_code][globals.simulation_optionright]
globals.positions

[]

In [10]:
# 目前欲實現進度: 
# 1. 在get_simulation_time抓到價平後下模擬單，並在達到停損條件後(a. 成交價 or b. 強制平倉時間)出掉 * b.Done * a. Done
# 2. 做最佳移動掛單 *已完成在cover.dynamic_price_adjustment 待測試第二順位
# 3. 完成模擬下單 * Done
# 4. cover那邊加一個控制參數 * Done
# 5. 可以儲存上次倉位

### 細節補充 ###
# 實單方面 口數是CALL PUT共用
# 實單方面的平倉 只要考慮賣出就好 因此沒有設ACTION參數
# 不論實單或模擬單 皆沒有考慮禮拜三下單狀況
# 研究一下lastcontractprice的內部資訊是甚麼 沒用就註明一下
'''
positions.fill_contract(option_code)
contract_snap = globals.api.snapshots([globals.contract])
print(contract_snap)
'''

'\npositions.fill_contract(option_code)\ncontract_snap = globals.api.snapshots([globals.contract])\nprint(contract_snap)\n'

01:56:00 時刻之價平和檔位及成交點數和:  TX214700 266.0
A deal has been detected.
Deal information: code:TX214700X2, action:Buy, price:157, quantity:5, optionright:P
Delivery month:202212, security type: OPT
***
A position with code=TX214700X2, type=Buy Put, quantity=5, price=157 has been recorded!

***

***
A position with code=TX214700X2, type=Buy Put, quantity=5, price=157 has been added to the track list!

***

A loss stop has been detected. Market price: 153.0, buy price: 157

A deal has been detected.
Deal information: code:TX214700X2, action:Sell, price:154, quantity:5, optionright:P
Delivery month:202212, security type: OPT
***
A position with code=TX214700X2, type=Sell Put, quantity=5, price=154 has been recorded!

***

